# Lab 7: OpenAPI로 전체 기업 고유 번호 가져오기

## 기업 고유번호 가져오기 - 샘플코드 테스트

In [ ]:
## 샘플 코드 결과를 실행해 PC에 저장하기
# requests로 샘플 코드 실행해 보기
import requests, json
import pandas as pd
from zipfile import ZipFile 

url=

response = requests.get(url)
response

In [ ]:
# parameter를 dict형식으로 변경하고 requests 실행
parameters = {

}
url = 

response = requests.get(url, params=parameters)
response


## 기업 고유번호 가져오기 - zip file을 가져와서 PC에 저장하기

In [ ]:
# response를 zip file로 저장하기

zip_file_name = '../working/corp_codes.zip'


In [ ]:
# 저장된 파일 확인
import os
os.listdir('../working')

In [2]:
# 참고: python변수를 그 모양 그대로 파일로 저장하기: pickle
import pickle

with open('../working/variable.pkl', 'wb') as file:
    pickle.dump(response, file, pickle.HIGHEST_PROTOCOL)

del response

with open('../working/variable.pkl', 'rb') as file:
    response = pickle.load(file)
    print(response)

{'a': 'b'}


## 기업 고유번호 가져오기 - 저장된 압축 파일을 압축해제하여 내용 확인하기

In [ ]:
## zip file에서 xml file을 데이터프레임으로 변환하기

# zipfile로 읽어 오기
zf = 

# zipfile내의 압축된 파일 가져오기
len(zf.filelist)

In [ ]:
# 압축된 파일의 원 이름 가져오기
zipped_file = zf.filelist[0].filename
zipped_file

In [ ]:
# 압축해제하고 파일의 내용 확인해 보기
print(zf.open(zipped_file).read().decode()[:500])

## 기업 고유번호 가져오기 - xml을 dict로 만든 후 데이터프레임으로 만들기

In [ ]:
# 기업 고유번호 데이터프레임 만들기
import xmltodict
xmltodict.parse(zf.open(zipped_file).read())

In [ ]:
# 정보 부분만 접근하기


In [ ]:
# 데이터프레임으로 변환하기
corp_list = pd.DataFrame( )
corp_list

In [ ]:
# 상장기업만 추출하기 --> stock_code가 있음
listed = 

In [ ]:
# 종목코드가 '005930'인 기업 정보 가져오기
listed.loc[]

In [ ]:
# 종목명에 '삼성'이 포함된 기업 정보 가져오기
listed.loc[]

In [ ]:
# 저장하기
listed.to_pickle('../working/listed.pkl')

## 기업 고유번호 가져오기 - DB에 저장하고 SQL로 문자열 검색하기

In [ ]:
# DB에 저장하기

from sqlalchemy import create_engine
engine = create_engine('sqlite:///../working/prices.db', echo=False)

In [ ]:
# listed --> 'listed' 테이블로 저장하기, 'replace'


In [ ]:
# 종목명에 '삼성'이 포함된 기업 정보 query해 오기

query = '''
'''

pd.read_sql_query(query, engine)

## 참고: ElementTree 이용 - xml 파일을 ET형식으로 변환하고 고유번호 확인

In [ ]:
## ElementTree 형식으로 변환해서 고유번호 등 확인하기
import xml.etree.ElementTree as ET

# xml tree 만들기
tree = ET.fromstring(zf.open(zipped_file).read())

# xml tree 내용 확인하기
tree.tag

In [ ]:
tree.getchildren()[:5]

In [ ]:
list(tree)[:5]

In [ ]:
# tree에서 첫번째 기업 고유번호 등 가져오기

tree.find('list')

In [ ]:
for element in tree.find('list'):
    print(element.tag, element.text)

## 참고: ElementTree 이용 - ET에서 처음 5개 기업 고유번호 가져오기

In [ ]:
# tree에서 처음 5개의 기업 고유번호 등 가져오기

tree.findall('list')[:5]

In [ ]:
for corp in tree.findall('list')[:5]:
    for element in corp:
        print(element.text, end=' ')
    print()

In [ ]:
## xml tree에서 기업 고유번호 등 추출해서 dataframe으로 변환하기

# tag 이름을 list에 담기
cols = []
for element in tree.find('list'):
    cols.append(element.tag)
cols

In [ ]:
# tag 안의 text들을 list에 담기
[element.text for element in tree.findall('list')[0]]

In [ ]:
# list comprehension으로 만들어 보기
[[element.text for element in corp] for corp in tree.findall('list')][:5]
corp_list = [[element.text for element in corp] for corp in tree.findall('list')]

In [ ]:
# list를 dataframe으로 변환하기
pd.DataFrame(corp_list, columns=cols)
df_corp = pd.DataFrame(corp_list, columns=cols)
df_corp

In [ ]:
# 종목코드가 '005930'인 기업 정보 가져오기
df_corp[]